 # Extra cleaning for all movies 

In [22]:
#Import libaries 
import pandas as pd
import re
import numpy as np 
import os

# set wd 
os.getcwd()
os.chdir('/Users/Sofie/Disney Thesis')

# import clean scripts
data = pd.read_pickle("scripts_clean_final.pickle")

#Brug Gender_Updated (as gender column)

In [23]:
# read in scripts with manually implemented changes after inspection of character names 
os.chdir('/Users/Sofie/Disney Thesis/Test_Scripts')
data['Clean_Script'][49] = pd.read_excel('Inside Out_updated.xlsx', index_col=0)
data['Clean_Script'][61] = pd.read_excel('Luca_updated.xlsx', index_col=0)

<ipython-input-23-cc77433a1f13>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Clean_Script'][49] = pd.read_excel('Inside Out_updated.xlsx', index_col=0)
<ipython-input-23-cc77433a1f13>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Clean_Script'][61] = pd.read_excel('Luca_updated.xlsx', index_col=0)


In [24]:
# wrong names which by mistake was not cleaned
del_name = ["you!</i><br/> lead cda agent", "finding dory logo.pngyoung dory", 
"always -- puts", "musician's blessing. we gotta find", "only de la cruz. if you can't help",
"tea rosita... and", "tea victoria... and those two", 
"of course you love mango! only a", "dad was a wizard!"]

names = []
# delete wrong character names if the names are in the above list
# The wrong character names were find based on a inspection of characters using
# len(row['Character'].split()) > 2 
for movie in data['Clean_Script']:
    for index, row in movie.iterrows():
        clean_names = []
        
        if row['Character_lower'] in del_name: 
            movie.drop(index, inplace=True)

In [25]:
# Replace &amp; in name 
# Replace spelling mistake in gender
names_l = []
names_c = []
genders = []
for index, row in data.iterrows():
    clean_names = []
    clean_names_c = []
    clean_gender = []
    for name in row['Clean_Script']['Character_lower']:
        name = re.sub("&amp;", "&", name)
        
        clean_names.append(name)
    names.append(clean_names)
    
    for name_c in row['Clean_Script']['Character']:
        name_c = re.sub("&amp;", "&", name_c)
        
        clean_names_c.append(name_c)
    names_c.append(clean_names_c)
    
    for gender in row['Clean_Script']['Gender_updated']:
        gender = re.sub("Famale", "Female", gender)
        gender = re.sub("Non-human", "Non-gendered", gender)
        clean_gender.append(gender)
    genders.append(clean_gender)
    
for i in range(len(data)):
    data['Clean_Script'][i]['Character_lower'] = names[i]
    data['Clean_Script'][i]['Character'] = names_c[i]
    data['Clean_Script'][i]['Gender_updated'] = genders[i]

In [26]:
# delete extra gender columns 
movies = []
for movie in data['Clean_Script']:
    movie = movie.drop(['Gender', 'Gender_new'], 1)
    movies.append(movie)

for i in range(len(data)):
    data['Clean_Script'] = movies

In [27]:
# Remove all empty lines 
for movie in data['Clean_Script']:
    for index, row in movie.iterrows():
        if len(row['Line']) < 2 and row['Line'] != "I":
            movie.drop(index, inplace=True)

In [28]:
# check if there is any empty lines 
for index, row in data.iterrows():
    if isinstance(row['Clean_Script'], pd.DataFrame):
        movie = row['Movie']
        for line in row['Clean_Script']['Line']:
            if len(line) < 2 and line != "I":
                print(movie, line)
                #row['Clean_Script'].drop(index, inplace=True)

In [29]:
# inspect all grouped charactes to see whether they are inter- or intra-gendered 
group_total = []
for index, row in data.iterrows():
    movie = row['Movie']
    group = []
    for _, r1 in row['Clean_Script'].iterrows():
        if r1['Gender_updated'] == "Group":
            #print(movie, r1['Character'])
            group.append("Group")
        elif "&" in r1['Character_lower'] or " and " in r1['Character_lower'] or "/" in r1['Character_lower']:
            group.append("Group")
        else:
            group.append("Single")
    group_total.append(group)

for i in range(len(data)):
    data['Clean_Script'][i]['Group'] = group_total[i]              

In [30]:
# charactes where the gender is descriped as transgendered, but should be single-gendered
female_group = ["Mia & Tia", "Anna & Elsa", "Honeymaren&Elsa"]
male_group = ["Mike and Sulley", "Sulley and Mike", "The Willie Brothers", 
              "TEO OSCAR/TEO FELIPE", "Mr. Incredible and Winston", "Frozone and Winston",
              "Woody and Bo Peep", "Bunny & Woody", "Woody & Buzz", "Ducky & Bunny", 
              "Woody & Bo"]
#empty list 
gender_group = []

#looping through all clean scripts, and if character is in one of the two lists^
#change the gender 
for index, row in data.iterrows():
    gender = []
    for _, r1 in row['Clean_Script'].iterrows():
        
        if r1['Character'] in male_group:
            gender.append('Male')
        elif r1['Character'] in female_group:
            gender.append('Female')
        else:
            gender.append(r1['Gender_updated'])
    gender_group.append(gender)

#append updated gender list to clean scripts 
for i in range(len(data)):
    data['Clean_Script'][i]['Gender_updated'] = gender_group[i]

In [31]:
# update olaf to be male
genders_ = []
for index, row in data.iterrows():
    genders = []
    for _, r1 in row['Clean_Script'].iterrows():
        if r1['Character_lower'] == "olaf":
            genders.append("Male") 
        elif r1['Character_lower'] == "taran":
            genders.append("Male")
        elif r1['Character_lower'] == "sulley":
            genders.append("Male")
        else:
            genders.append(r1['Gender_updated'])
           
    genders_.append(genders)
        #if r1['Character'] in male_group:
for i in range(len(data)):
    data['Clean_Script'][i]['Gender_updated'] = genders_[i]

In [32]:
lines_lionking = []
for line in data['Clean_Script'][22]['Line']:
    line = line.split("-=-=-")[0]
    lines_lionking.append(line)
    
data['Clean_Script'][22]['Line'] = lines_lionking

In [33]:
# save data frame
os.chdir('/Users/Sofie/Disney Thesis')
data.to_pickle("scripts_clean_final_22mar.pickle")

In [34]:
data = pd.read_pickle("scripts_clean_final_22mar.pickle")